# Covid-19 Data

In [ ]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

## Load the Data

For cases:

In [ ]:
covid_case_data_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
case_data = pd.read_csv(covid_case_data_url)
case_data

For deaths:

In [ ]:
covid_death_data_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
death_data = pd.read_csv(covid_death_data_url)
death_data

## Collect Data of selected Countries

In [ ]:
def select_data(data, selected_countries):
    days = list(data.columns[4:])
    values = {"Dates": days}
    selected_data = pd.DataFrame(values, columns = ["Dates"])
    selected_data['Days'] = pd.to_datetime(selected_data['Dates'], format='%m/%d/%y')
    selected_data.drop('Dates',axis=1,inplace=True)
    vals = data.loc[case_data['Country/Region'].isin(selected_countries)].drop(['Province/State','Long','Lat'],axis='columns')
    vals = vals.groupby('Country/Region').sum().transpose()
    vals.reset_index(drop=True, inplace=True)
    res = pd.concat([selected_data,vals],axis=1)
    res.set_index('Days',inplace=True)
    return res


country_selector = widgets.SelectMultiple(
    options=sorted(set(case_data['Country/Region'])),
    value=['Germany'],
    rows=10,
    description='Countries:',
    disabled=False
)

button = widgets.Button(
    description='Select Data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to start selection of data',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

#selected_countries = {'Germany','Australia'}
case_sel_data = 0
death_sel_data = 0

def start_selection(a):
    countries = set(country_selector.value)
    global case_sel_data
    case_sel_data = select_data(case_data, countries)
    global death_sel_data
    death_sel_data= select_data(death_data, countries)

button.on_click(start_selection)

display(country_selector)
display(button)

In [ ]:
case_sel_data

In [ ]:
death_sel_data

## Plot the selected Data

In [ ]:
def plot_selected_time_range(date_range):
    start,end = date_range
    sub = case_sel_data[start:end]
    fig, ax =plt.subplots(2,1)
    sns.lineplot(data = sub, dashes=False,ax=ax[0])
    sns.lineplot(data = sub.diff(), dashes=False,ax=ax[1])
    fig.suptitle("Covid19 Cases")
    fig.show()

options = [(i.strftime(' %d %b %y '), i) for i in case_sel_data.index]
slider = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(case_sel_data.index)-1),
    description='Range:',
    disabled=False
)

widgets.interactive(plot_selected_time_range, date_range = slider)

In [ ]:
def plot_selected_time_range(date_range):
    start,end = date_range
    sub = death_sel_data[start:end]
    fig, ax =plt.subplots(2,1)
    sns.lineplot(data = sub, dashes=False,ax=ax[0])
    sns.lineplot(data = sub.diff(), dashes=False,ax=ax[1])
    fig.suptitle("Covid19 Death Rates")
    fig.show()

options = [(i.strftime(' %d %b %y '), i) for i in death_sel_data.index]
slider = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(death_sel_data.index)-1),
    description='Range:',
    disabled=False
)

widgets.interactive(plot_selected_time_range, date_range = slider)

## Save the Data of the last Day for selected Countries

If you run the notebook locally

In [ ]:
def save_last_day_data(path, kind, data):
    last_day = data.index[-1].strftime('%d-%b-%Y')
    last_day_data_dict =  {
        f'Cum {last_day}': data.iloc[-1],
        f'Diff {last_day}': data.diff().iloc[-1]
    }
    last_day_data = pd.DataFrame(last_day_data_dict)
    writer = pd.ExcelWriter(f'{path}/Covid19_Data_{kind}_{last_day}.xlsx', engine='xlsxwriter')
    last_day_data.to_excel(writer, sheet_name='Sheet1')
    writer.save()

path = input("Path to save the File (C:/path/to/target/folder):")
if path != '':
    save_last_day_data(path, "Cases", case_sel_data)
    save_last_day_data(path, "Deaths", death_sel_data)

## Create Download-Links for the Data of the last Day for selected Countries

If you run the notebook via Binder

In [ ]:
from IPython.display import HTML
import io
import base64

def get_df_last_day_data(data):
    last_day = data.index[-1].strftime('%d-%b-%Y')
    last_day_data_dict =  {
        f'Cum {last_day}': data.iloc[-1],
        f'Diff {last_day}': data.diff().iloc[-1]
    }
    last_day_data = pd.DataFrame(last_day_data_dict)
    return last_day_data

def create_download_link_excel( df, title = "Download Excel file", filename = "data.xls"):
    output = io.BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()
    xlsx_data = output.getvalue()
    b64 = base64.b64encode(xlsx_data)
    payload = b64.decode()
    html = '<a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

HTML("""
<p> Download of selected Data of the last Day</p>
<table>
  <tr>
    <th></th> <th>Cases</th> <th>Deaths</th>
  </tr>
  <tr>
    <td>CSV:</td> <td>{csv_cases}</td> <td>{csv_deaths}</td>
  </tr>
  <tr>
    <td>XLSX:</td> <td>{xlsx_cases}</td> <td>{xlsx_deaths}</td>
  </tr>
</table>""".format(csv_cases = create_download_link(get_df_last_day_data(case_sel_data), "Cases CSV", "cases_data.csv"),
                   xlsx_cases = create_download_link_excel(get_df_last_day_data(case_sel_data), "Cases XLSX", "cases_data.xlsx"),
                   csv_deaths = create_download_link(get_df_last_day_data(death_sel_data), "Deaths CSV", "deaths_data.csv"),
                   xlsx_deaths = create_download_link_excel(get_df_last_day_data(death_sel_data), "Deaths XLSX", "deaths_data.xlsx")
                  ))